In [41]:
%load_ext autoreload
%reload_ext autoreload
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
%jaxtyping.typechecker beartype.beartype

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [1]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

from jaxtyping import Float, Array

from n3.architecture.controller import IdentityController, ControllerLike
from n3.architecture.model import N3, ModelLike
from n3.data import bessel
from n3.utils.utils import grad_norm

### Config

In [25]:
n_samples = 2**15
test_size = 0.2

seed = 0
key = jax.random.PRNGKey(seed)
N_max = 10  # per layer max number of neurons
size_influence = 0.32
epochs = 5_000
print_every = 100

### Dataset

In [26]:
x_train, x_test, y_train, y_test = bessel.generate_data(
    n_samples=n_samples,
    test_size=test_size,
    scaler=MinMaxScaler(feature_range=(-1, 1)),
    seed=seed,
)

In [ ]:
plt.plot(x_train, y_train, "k.", alpha=0.5, label=f"train:{len(x_train)}")
plt.plot(x_test, y_test, "r.", alpha=0.3, label=f"test:{len(x_test)}")
plt.legend()

### Training scheme

In [28]:
def compute_base_loss(
    model: ModelLike,
    control: ControllerLike,
    x: Float[Array, "batch 1"],
    y: Float[Array, "batch 1"],
) -> Float[Array, ""]:
    pred = jax.vmap(model, in_axes=(0, None))(x, control)
    return jnp.mean((pred - y) ** 2)

In [29]:
def compute_size_loss(controller: ControllerLike) -> Float[Array, ""]:
    N = controller(jnp.ones((1,)))
    return size_influence * jnp.mean((N - 1.0) ** 2)

In [30]:
@eqx.filter_jit
def make_step(
    model: ModelLike,
    controller: ControllerLike,
    x: Float[Array, "batch 1"],
    y: Float[Array, "batch 1"],
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> tuple[Float[Array, ""], ModelLike, ControllerLike, optax.OptState]:
    loss_base, grads_base = eqx.filter_value_and_grad(compute_base_loss)(
        model, controller, x, y
    )
    loss_size, grads_size = eqx.filter_value_and_grad(compute_size_loss)(controller)
    loss = loss_base + loss_size

    updates, opt_state = optim.update([grads_base, grads_size], opt_state)

    model = eqx.apply_updates(model, updates[0])  # type: ignore
    controller = eqx.apply_updates(controller, updates[1])  # type: ignore
    return loss, model, controller, opt_state

In [31]:
@eqx.filter_jit
def test_step(
    model: ModelLike,
    controller: ControllerLike,
    x: Float[Array, "batch 1"],
    y: Float[Array, "batch 1"],
) -> Float[Array, ""]:
    return compute_base_loss(model, controller, x, y) + compute_size_loss(controller)

In [33]:
model_key, control_key = jax.random.split(key)
n3 = N3(1, 1, [N_max], model_key)
control = IdentityController(1, control_key)

optim = optax.adam(learning_rate=1e-3)
opt_state = optim.init(eqx.filter([n3, control], eqx.is_inexact_array))

In [ ]:
epoch_list = []
test_losses = []
train_losses = []
controls2 = []
base_grad_norms = []
control_grad_norms = []

for epoch in range(epochs):
    train_loss, n3, control, opt_state = make_step(
        n3, control, x_train, y_train, optim, opt_state
    )

    if epoch % print_every == 0:
        epoch_list.append(epoch)
        test_loss = test_step(n3, control, x_test, y_test)

        test_losses.append(test_loss)
        train_losses.append(train_loss)
        controls2.append(control.params.item() ** 2)
        control_grad_norms.append(
            grad_norm(eqx.filter_grad(compute_size_loss)(control))
        )
        print(
            f"epoch: {epoch_list[-1]}, train_loss: {train_losses[-1]:.4e}, test_loss: {test_losses[-1]:.4e} control2: {controls2[-1]:.4e}"
        )
        print(f"Control_grad_norm: {control_grad_norms[-1]:.4e}")

In [ ]:
fig, ax1 = plt.subplots()

# Plotting train and test losses
ax1.plot(epoch_list, train_losses, "b.", label="Train Loss")
ax1.plot(epoch_list, test_losses, "r.", label="Test Loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss", color="k")
ax1.set_yscale("log")
# ax1.set_xscale("log")

# Creating a second y-axis to plot control gradient norms
ax2 = ax1.twinx()
ax2.plot(epoch_list, controls2, "g.", label="Control^2")
ax2.set_ylabel("Control^2", color="g")

ax1.legend()
ax2.legend()
plt.show()

In [ ]:
y_pred = jax.vmap(n3, in_axes=(0, None))(x_test, control)
test_loss = jnp.mean((y_pred - y_test) ** 2)

plt.plot(x_test, y_test, "k.", alpha=0.2, label="truth")
plt.plot(x_test, y_pred, "r.", alpha=0.2, label=f"MSE: {test_loss: .4e}")
plt.legend()